In [ ]:
import json

def parse_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # 打印整个JSON结构
    print(json.dumps(data, indent=4))

    # 进行其他操作，例如提取特定字段、遍历数据等

# 替换为你的JSON文件路径
json_file_path = '/home/chenzhixuan/Workspace/R2Gen/data/iu_xray/annotation.json'

parse_json_file(json_file_path)

# 统计数据集元素个数

In [ ]:
import json

def count_elements_in_json(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)
        train_count = len(data['train'])
        val_count = len(data['val'])
        test_count = len(data['test'])
    return train_count, val_count, test_count

json_file_path = '/jhcnas1/chenzhixuan/mimic-cxr-jpg/annotation.json'
train_count, val_count, test_count = count_elements_in_json(json_file_path)
print('train_count: ', train_count)
print('val_count: ', val_count)
print('test_count: ', test_count)

# 测试DataParallel

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import DataParallel
from torch.utils.data import DataLoader
import torchvision.models as models
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

# 1. 定义模型
class Child_MyModel(nn.Module):
    def __init__(self):
        super(Child_MyModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x, y):
        return self.fc(x+y)

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(10, 1)
        self.model = models.resnet50(pretrained=True)

    def forward(self, x):
        return self.model(x)

# 2. 封装模型
device_ids = [0, 1]  # 使用编号为 0 和 1 的 GPU 设备
model = MyModel()
model.cuda()
model = DataParallel(model, device_ids=device_ids)

# 4. 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 5. 训练模型
for epoch in range(10):  # 迭代 10 次
    for i in range(100):  # 每次迭代有 100 个批次
        # 生成随机输入数据和标签
        inputs = torch.randn(2, 3, 224, 224).to('cuda:0')  # 假设每个批次有 32 个样本，每个样本是 3x224x224 的图像
        labels = torch.randn(2, 1000).to('cuda:0')  # 假设每个样本有 1000 个类别

        # 清零梯度
        optimizer.zero_grad()

        # 正向传播
        outputs = model(inputs)

        # 计算损失
        loss = criterion(outputs, labels)

        # 反向传播
        loss.backward()

        # 更新权重
        optimizer.step()

        # 打印损失
        if i % 10 == 0:
            print(f'Epoch {epoch}, Batch {i}, Loss: {loss.item()}')



In [4]:
from models.r2gen import R2GenModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import DataParallel
from torch.utils.data import DataLoader
import torchvision.models as models
import argparse
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"


# 2. 封装模型
device_ids = [0, 1]  # 使用编号为 0 和 1 的 GPU 设备
model = R2GenModel(0,0)
model.cuda()
model = DataParallel(model, device_ids=device_ids)

# 4. 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 5. 训练模型
for epoch in range(10):  # 迭代 10 次
    for i in range(100):  # 每次迭代有 100 个批次
        # 生成随机输入数据和标签
        inputs = torch.randn(5, 2, 3, 224, 224).to('cuda:0')  # 假设每个批次有 32 个样本，每个样本是 3x224x224 的图像
        labels = torch.randn(2, 1000).to('cuda:0')  # 假设每个样本有 1000 个类别

        # 清零梯度
        optimizer.zero_grad()

        # 正向传播
        outputs = model(inputs, labels, mode='train')

        # 计算损失
        loss = criterion(outputs, labels)

        # 反向传播
        loss.backward()

        # 更新权重
        optimizer.step()

        # 打印损失
        if i % 10 == 0:
            print(f'Epoch {epoch}, Batch {i}, Loss: {loss.item()}')



/home/chenzhixuan/anaconda3/envs/r2gen/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/chenzhixuan/anaconda3/envs/r2gen/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


/home/chenzhixuan/anaconda3/envs/r2gen/lib/python3.9/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


KeyboardInterrupt: 